In [2]:
import os 
import re
import keras

import numpy as np
import pandas as pd
import matplotlib.pyplot as plt

from keras.models import Sequential
from keras.layers import Dense, Dropout, Embedding, LSTM, SpatialDropout1D
from keras.callbacks import ModelCheckpoint
from keras.preprocessing.text import Tokenizer
from keras.preprocessing.sequence import pad_sequences

from sklearn.metrics import roc_auc_score
from sklearn.model_selection import train_test_split

#Based on example here: https://devklaus.wordpress.com/2018/03/11/sentiment-analysis-on-us-twitter-airlines-dataset-a-deep-learning-approach/

Using TensorFlow backend.


In [3]:
train = pd.read_csv('../data/train.csv',sep=';',index_col=0)
val = pd.read_csv('../data/val.csv',sep=';',index_col=0)
test = pd.read_csv('../data/test.csv',sep=';',index_col=0)

In [58]:

max_features = 2000
tokenizer = Tokenizer(num_words=max_features, split=' ')
tokenizer.fit_on_texts(train['text'].values.tolist() + val['text'].values.tolist() + test['text'].values.tolist())


X = tokenizer.texts_to_sequences(train['text'].values.tolist() + val['text'].values.tolist() + test['text'].values.tolist())
X = pad_sequences(X)
Xtrain = X[:len(train)]
Xval = X[len(train):len(train)+len(val)]
Xtest = X[len(train)+len(val):]

In [55]:
(train['text'].values.tolist() + val['text'].values.tolist() + test['text'].values.tolist())

"@SouthwestAir you're my early frontrunner for best airline! #oscars2016"

In [59]:
embed_dim = 128
lstm_out = 196

model = Sequential()
model.add(Embedding(max_fatures, embed_dim,input_length = Xtrain.shape[1]))
model.add(Dropout(0.5))
model.add(LSTM(lstm_out, dropout=0.2, recurrent_dropout=0.2))
model.add(Dense(3,activation='softmax'))
model.compile(loss = 'categorical_crossentropy', optimizer='adam',metrics = ['accuracy'])
print(model.summary())

_________________________________________________________________
Layer (type)                 Output Shape              Param #   
embedding_9 (Embedding)      (None, 33, 128)           256000    
_________________________________________________________________
dropout_9 (Dropout)          (None, 33, 128)           0         
_________________________________________________________________
lstm_9 (LSTM)                (None, 196)               254800    
_________________________________________________________________
dense_9 (Dense)              (None, 3)                 591       
Total params: 511,391
Trainable params: 511,391
Non-trainable params: 0
_________________________________________________________________
None


In [60]:
ytrain = pd.get_dummies(train['sentiment']).values
yval = pd.get_dummies(val['sentiment']).values
ytest = pd.get_dummies(test['sentiment']).values
print(Xtrain.shape,ytrain.shape)
print(Xval.shape,yval.shape)
print(Xtest.shape,ytest.shape)

(10000, 33) (10000, 3)
(2000, 33) (2000, 3)
(2640, 33) (2640, 3)


In [62]:
batch_size = 512
history = model.fit(Xtrain, ytrain, epochs = 10, batch_size=batch_size, validation_data=(Xval, yval))

Train on 10000 samples, validate on 2000 samples
Epoch 1/10
10000/10000 [==============================] - 18s 2ms/step - loss: 0.3375 - acc: 0.8700 - val_loss: 0.5758 - val_acc: 0.7855
Epoch 2/10
10000/10000 [==============================] - 18s 2ms/step - loss: 0.3274 - acc: 0.8699 - val_loss: 0.6019 - val_acc: 0.7855
Epoch 3/10
10000/10000 [==============================] - 18s 2ms/step - loss: 0.3189 - acc: 0.8760 - val_loss: 0.6187 - val_acc: 0.7875
Epoch 4/10
10000/10000 [==============================] - 18s 2ms/step - loss: 0.3128 - acc: 0.8778 - val_loss: 0.6223 - val_acc: 0.7820
Epoch 5/10
10000/10000 [==============================] - 18s 2ms/step - loss: 0.2994 - acc: 0.8844 - val_loss: 0.6364 - val_acc: 0.7785
Epoch 6/10
10000/10000 [==============================] - 18s 2ms/step - loss: 0.2926 - acc: 0.8831 - val_loss: 0.6303 - val_acc: 0.7795
Epoch 7/10
10000/10000 [==============================] - 18s 2ms/step - loss: 0.2868 - acc: 0.8906 - val_loss: 0.6365 - val_acc:

In [23]:
Xtrain

array([], shape=(10000, 0), dtype=int32)